In [1]:
import pandas as pd
import numpy as np
import calendar
import os

from datetime import datetime
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import streaming_bulk
from es_pandas import es_pandas

# 导入datetime库
import datetime
import random
#import modin.pandas as mpd

#import commond.ipynb from same folder
import import_ipynb
from common import *

importing Jupyter notebook from common.ipynb


In [2]:
# generate nums of rows in dataframe
def random_dataframe(nums):
    #using numpy's randint
   df = pd.DataFrame(np.random.randint(0,nums,size=(15, 41)), columns=list(get_interests_columns()))
   return df

ds = random_dataframe(100)
ds


,Date,Time,Comm Status,CraneNumber,Weight,Frequency,Spreader_20FT,Spreader_40FT,Spreader_45FT,Spreader_Twin,...,OT_Control STS_Gantry Right (Controller),OT_Control STS_Gantry Left (Controller),OT_Control STS_Boom Up (Controller),OT_Control STS_Boom Down (Controller),Trolley Position,Load Cell1 Data,Load Cell2 Data,Load Cell3 Data,Load Cell4 Data,SP STS_Landed
0,93,33,32,91,81,42,94,39,36,30,...,69,13,0,54,36,85,98,34,78,99
1,78,92,91,0,90,28,82,20,46,4,...,83,98,90,13,44,53,0,97,37,80
2,99,32,96,87,49,66,58,89,94,49,...,51,28,46,19,77,59,2,22,58,83
3,23,80,23,74,46,80,13,75,85,11,...,14,94,31,65,77,22,91,26,61,82
4,63,5,53,51,31,10,89,22,82,67,...,67,5,50,21,21,41,94,51,59,91
5,78,24,80,18,54,9,53,95,31,56,...,47,20,76,96,89,8,33,95,9,94
6,65,31,94,21,22,38,12,38,49,10,...,46,85,17,10,86,23,11,45,32,94
7,56,79,66,59,84,79,1,81,29,96,...,90,71,62,41,15,13,67,40,44,8
8,25,85,83,75,21,19,99,75,23,51,...,71,25,69,71,21,94,6,9,15,22
9,59,88,99,76,78,64,81,16,59,49,...,68,7,51,79,30,82,34,84,74,96


In [3]:
def callback_ingestion(data_set):
    # Information of es cluseter
    es_host = 'localhost:9200'
    index = 'wholeyear_daq_index'

    # crete es_pandas instance
    ep = es_pandas(es_host)
    # init template if you want
    doc_type = 'records'

    ep.init_es_tmpl(data_set, doc_type)

    # Example of write data to es, use the template you create
    ep.to_es(data_set, index, doc_type=doc_type, thread_count=4, chunk_size=7200)


In [7]:
def print_time(msg):
    # datetime object containing current date and time
    now = datetime.datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%Y-%m-%d %H:%M:%S.%f%Z")
    print(msg+dt_string)

In [16]:
# return dataframe by timerange
def ingest_bulk_dataframe(start_date, end_date, crane_number, callback=None):
    # 获取时间间隔
    delta = end_date - start_date
    
    ds = pd.DataFrame()
    const_ds = pd.DataFrame()

    const_ds.at[0, "CraneNumber"] = crane_number
    const_ds.at[1, "CraneNumber"] = crane_number
    const_ds.at[0, "Twistlock"] = 0
    const_ds.at[1, "Twistlock"] = 1
    const_ds.at[0, "Comm Status"] = 'OK'
    const_ds.at[1, "Comm Status"] = 'False'
    const_ds.at[0, "Asset Type"] = 'QC'
    const_ds.at[1, "Asset Type"] = 'YC'
    
    col_list = get_interests_columns_simple()
    # 遍历获得每一个时间点
    for i in range(int(delta.total_seconds())+1):
        #print_time("randint")
        ss = random.randint(10, 100)
        temp = start_date + datetime.timedelta(seconds=i)
        dt1 = temp + datetime.timedelta(milliseconds=ss)
        dt2 = dt1 + datetime.timedelta(milliseconds=500)
        
        #print_time("start create df ")
        df = pd.DataFrame(np.random.randint(10,1000,size=(2, len(col_list))), columns=list(col_list))
        #print_time("finish create df ")
        
        df.at[0, "timestamp"] = dt1.astimezone().isoformat()
        df.at[1, "timestamp"] = dt2.astimezone().isoformat()

        #print_time("finish add df ")
        new_df = pd.concat([df, const_ds], axis=1)
        del df
        #print_time("finish cancat df ")
        ds = pd.concat([ds, new_df], axis=0)
        #ds = ds.append(new_df)
        #print(ds)
        #print_time("finish append df ")
        
        if ds.shape[0] >= 14400:
           #print_time("start ingestion to ES ")
           callback(ds)
           print_time("finish ingestion to ES ")
           msg = dt2.strftime("%d-%b-%Y (%H:%M:%S.%f)")
           print('Ingest %s  is completed!' % msg)
           ds = ds[0:0]
           
    if ds.shape[0] > 0:
       callback(ds) 


In [ ]:
# ingest whole year data for all crane
for i in range(1):
    crane = 'PQC' + str(random.randint(500, 900))
    # 创建开始时间点和结束时间点
    dt_start = datetime.datetime(2020,1,1,0,0,1)
    dt_end =   datetime.datetime(2021,1,1,0,0,0)
    ingest_bulk_dataframe(dt_start, dt_end, crane, callback_ingestion)
    print('Ingest %s  is completed!' % crane)

 93% (13489 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-05T20:23:17.336252
Ingest 01-Jan-2020 (02:00:00.550000)  is completed!


 89% (12942 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-05T20:23:46.964032
Ingest 01-Jan-2020 (04:00:00.594000)  is completed!


 98% (14218 of 14400) |################# | Elapsed Time: 0:00:00 ETA:   0:00:00

...

finish ingestion to ES 2020-07-05T20:56:20.148451
Ingest 06-Jan-2020 (18:00:00.578000)  is completed!


 89% (12942 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-05T20:56:48.980906
Ingest 06-Jan-2020 (20:00:00.513000)  is completed!


 91% (13125 of 14400) |################  | Elapsed Time: 0:00:00 ETA:   0:00:00

finish ingestion to ES 2020-07-05T20:57:18.904119
Ingest 06-Jan-2020 (22:00:00.528000)  is completed!


In [53]:
dt_start = datetime.datetime(2017,1,1,0,0,1)
dt_end =   datetime.datetime(2017,12,31,0,4,2)

delta = dt_end - dt_start
print_time(str(delta.days*24*60*60+delta.seconds+1))
print_time(str(delta.total_seconds()))


31449842 04/07/2020 22:00:51.846222
31449841.0 04/07/2020 22:00:51.846875


In [7]:
def get_day_time_range(start_day, end_day):
    # 获取时间间隔
    delta = start_day - start_day
    ds = pd.DataFrame()
    
    for i in range(delta.days+1):
        df.at[0, "Start"] = start_day.date